# Langchain (v0.3) 의 주요 Components
https://python.langchain.com/docs/how_to/#components

1. Chat models
- Chat model 은 '메시지'를 입력받아 출력하는 형태의 언어 모델. 

1. Messages

1. Prompt templates

1. Example selectors

1. LLMs

1. Output parsers

1. Document loaders

1. Text splitters

1. Embedding models

1. Vector stores

1. Retrievers


1. Chains
https://python.langchain.com/docs/versions/migrating_chains/


1. Agents

1. Memory

1. Callbakcs


# prompts
https://python.langchain.com/api_reference/core/prompts.html#module-langchain_core.prompts

In [1]:
"""
prompts class 계층도

BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate
"""
None

# API Key 가져오기

In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
print(f'{os.environ['OPENAI_API_KEY'][:20]}...')

sk-proj-iKU13YeoxNgF...


# 1. FewShotPromptTemplate

In [5]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 0.PromptTemplate

In [7]:
# 방식1
prompt = PromptTemplate.from_template("What is the capital of {country}")
prompt.format(country='France')

'What is the capital of France'

In [8]:
# 템플릿을 만드는 이점중 하나는 '유효성 검사' 를 할 수 있다는 점이다
# 가령 아래와 같이 하면 에러다

# t.format()  # country= 를 빼먹으면 KeyError: 'country'

In [9]:
# 템플릿을 만드는 이점 또 한가지!
# prompt template 을 디스크에 '저장'하고 'load' 할 수 있기 떼문이다.

# 나중에 LLM 다룰때 prompt 는 매우 중요합니다.
# 대규모 프로젝트에서는 
# - 'prompt 만 만드는 팀'이 있고
# - '코딩하는 팀'이 따로 있을것이다.  
#    데이터베이스나 파일 등에 만들어 저장해 놓은 prompt 를 load 해야 할겁니다

In [10]:
# 방식2
prompt2 = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=['country'],
)

prompt2.format(country='Korea')

'What is the capital of Korea'

## 1.FewShotPromptTemplate
모델에 예제(sample) 주기

In [11]:
# 모델에게 '어떻게 대답해야 하는 지에 대한 예제(example)'를 AI 모델에게 주는 것이
# prompt 를 사용해서 '어떻게 대답해야 하는지 알려주는 것'보다 훨씬 좋다

# FewShotPromptTemplate 이 하는 일이 바로 그거다!
# - 이를 통해 예제(샘플)를 형식화(포맷) 할수 있다.
# - 이런 예제들을 데이터베이스등에 저장시켜놓고 활용할수도 있다

In [12]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate

In [13]:
# 모델이 나에게 '이런 식으로 답변해 줬으면 좋겠다' 라고 제시하는 example(예제) 들
examples = [
    {
        "question": "What do you know about France?",
        # ↓ 원하는 형식의 답변.
        "answer": """
              Here is what I know:
              Capital: Paris
              Language: French
              Food: Wine and Cheese
              Currency: Euro
              """,
    },
   {
    "question": "What do you know about Italy?",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "question": "What do you know about Greece?",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },   
]

In [14]:
# 일단, 예제(example) 없이 전달하면?
chat.invoke("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people and is known for its high standard of living.

French is the official language of France, and the country is known for its contributions to art, literature, philosophy, and science. France has a strong tradition of winemaking and is famous for its wine regions such as Bordeaux, Burgundy, and Champagne.

France is a founding member of the United Nations, NATO, and the European Union. It has a long history of colonialism and imperialism, with territories in Africa, the Caribbean, and the Pacific.

Overall, France is a diverse and culturally rich country with a significant influence on global politics, culture, and 

AIMessage(content='France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people and is known for its high standard of living.\n\nFrench is the official language of France, and the country is known for its contributions to art, literature, philosophy, and science. France has a strong tradition of winemaking and is famous for its wine regions such as Bordeaux, Burgundy, and Champagne.\n\nFrance is a founding member of the United Nations, NATO, and the European Union. It has a long history of colonialism and imperialism, with territories in Africa, the Caribbean, and the Pacific.\n\nOverall, France is a diverse and culturally rich country with a significant influence on glo

In [15]:
example_template = """
    Human: {question},
    AI: {answer},
"""
# ↑ {question} 과 {answer} 는 위 샘플과 동일한 key를 사용하여 작성한다.
None

In [16]:
example_prompt = PromptTemplate.from_template(example_template)
example_prompt

PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question},\n    AI: {answer},\n')

In [17]:
print(example_prompt.format(**examples[2]))


    Human: What do you know about Greece?,
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      ,



In [18]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,  # 사용할 prompt
    examples = examples,  # 준비된 예제들

    # suffix=
    # 포맷팅된 모든 예제 마지막에 나오는 내용.
    suffix="Human: What do you know abount {country}?",

    # 어떤 변수를 suffix 에서 사용할지 지정
    input_variables=['country']
)

prompt

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n              Here is what I know:\n              Capital: Paris\n              Language: French\n              Food: Wine and Cheese\n              Currency: Euro\n              '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question},\n    AI: {answer},\n'), suffix='Human: What do you know abount {country}?')

In [19]:
print(prompt.format(country='Germany'))


    Human: What do you know about France?,
    AI: 
              Here is what I know:
              Capital: Paris
              Language: French
              Food: Wine and Cheese
              Currency: Euro
              ,



    Human: What do you know about Italy?,
    AI: 
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      ,



    Human: What do you know about Greece?,
    AI: 
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      ,


Human: What do you know abount Germany?


In [20]:
# step1  example 리스트를 만들고   examples
# step2  FewShotPromptTemplate 에 전달했고 examples=
# step3  어떻게 전달한 예제들을 형식화 할지 알려주었고
# step4  마지막에 질문을 포함시켰다.  suffix, input_variables

# AI 는 우리의 예제들과 똑같은 구조, 형태로 답변하게 될겁니다

In [21]:
chain = prompt | chat

chain

FewShotPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, examples=[{'question': 'What do you know about France?', 'answer': '\n              Here is what I know:\n              Capital: Paris\n              Language: French\n              Food: Wine and Cheese\n              Currency: Euro\n              '}, {'question': 'What do you know about Italy?', 'answer': '\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      '}, {'question': 'What do you know about Greece?', 'answer': '\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      Currency: Euro\n      '}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n    Human: {question},\n    AI: {answer},\n'), suffix='Human: What do you know abount {country}?')
| ChatOpenAI(callbacks=[<langchain_core.callbacks.stream

In [22]:
chain.invoke({
    'country': "Germany"
})

AI: 
      I know this:
      Capital: Berlin
      Language: German
      Food: Bratwurst and Sauerkraut
      Currency: Euro

AIMessage(content='AI: \n      I know this:\n      Capital: Berlin\n      Language: German\n      Food: Bratwurst and Sauerkraut\n      Currency: Euro', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--55623dd3-5da9-4a03-a42d-3e16c77a4a05-0', usage_metadata={'input_tokens': 156, 'output_tokens': 35, 'total_tokens': 191, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
chain.invoke({
    'country': "Turkey"
})

AI: 
      I know this:
      Capital: Ankara
      Language: Turkish
      Food: Kebab and Baklava
      Currency: Turkish Lira

AIMessage(content='AI: \n      I know this:\n      Capital: Ankara\n      Language: Turkish\n      Food: Kebab and Baklava\n      Currency: Turkish Lira', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--47f183ab-d14d-49ef-b194-f10c1c170024-0', usage_metadata={'input_tokens': 156, 'output_tokens': 35, 'total_tokens': 191, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 2.FewShowChatMessagePromptTemplate

In [24]:
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

from langchain_core.prompts.chat import ChatPromptTemplate

In [25]:
# 모델이 나에게 '이런 식으로 답변해 줬으면 좋겠다' 라고 제시하는 example(예제) 들
examples = [
    {
        "country": "France",
        "answer": """
              Here is what I know:
              Capital: Paris
              Language: French
              Food: Wine and Cheese
              Currency: Euro
              """,
    },
   {
    "country": "Italy",
    "answer": """
      I know this:
      Capital: Rome
      Language: Italian
      Food: Pizza and Pasta
      Currency: Euro
      """,
  },
  {
    "country": "Greece",
    "answer": """
      I know this:
      Capital: Athens
      Language: Greek
      Food: Souvlaki and Feta Cheese
      Currency: Euro
      """,
  },   
]

In [26]:
prompt = ChatPromptTemplate.from_messages([
    ('human', "What do you know about {country}?"),  # 반드시 example 과 동일한 key 로!
    ('ai', "{answer}"),  # ai 의 답변들을 여기에 넣어줄거다.
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = prompt,
    examples = examples
    # suffix= 필요 없다.
)

final_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a geography expert, you give short answers."),
    example_prompt,
    ('human', 'What do you know about {country}?'),  # 질문
])


In [27]:
final_prompt.format_messages(country='Germany')

[SystemMessage(content='You are a geography expert, you give short answers.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know about France?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n              Here is what I know:\n              Capital: Paris\n              Language: French\n              Food: Wine and Cheese\n              Currency: Euro\n              ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know about Italy?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      I know this:\n      Capital: Rome\n      Language: Italian\n      Food: Pizza and Pasta\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What do you know about Greece?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\n      I know this:\n      Capital: Athens\n      Language: Greek\n      Food: Souvlaki and Feta Cheese\n      C

In [28]:
chain = final_prompt | chat

In [29]:
chain.invoke({'country': "Finland"})


      I know this:
      Capital: Helsinki
      Language: Finnish, Swedish
      Food: Rye bread and salmon
      Currency: Euro
      

AIMessage(content='\n      I know this:\n      Capital: Helsinki\n      Language: Finnish, Swedish\n      Food: Rye bread and salmon\n      Currency: Euro\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--cc034bf8-0d71-43b6-9d5b-98c69eb8baeb-0', usage_metadata={'input_tokens': 167, 'output_tokens': 33, 'total_tokens': 200, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [30]:
chain.invoke({'country': "South Korea"})


      I know this:
      Capital: Seoul
      Language: Korean
      Food: Kimchi and Bibimbap
      Currency: South Korean Won
      

AIMessage(content='\n      I know this:\n      Capital: Seoul\n      Language: Korean\n      Food: Kimchi and Bibimbap\n      Currency: South Korean Won\n      ', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--b083aa50-a553-4271-830a-d3309670339c-0', usage_metadata={'input_tokens': 168, 'output_tokens': 34, 'total_tokens': 202, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# 3.ExampleSelector

## LengthBasedExampleSelector

In [31]:
from langchain_core.example_selectors.length_based import LengthBasedExampleSelector

In [32]:
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [33]:
example_prompt = PromptTemplate.from_template("Human: {country}\nAI:{answer}")
example_prompt

PromptTemplate(input_variables=['answer', 'country'], input_types={}, partial_variables={}, template='Human: {country}\nAI:{answer}')

In [34]:
example_selector = LengthBasedExampleSelector(
    examples=examples, 
    example_prompt=example_prompt,  # 포맷팅 되는 양이 얼마나 되는지 알아보기 위해선 example_prompt 필요 
    max_length=10,  # 예제의 양을 얼마나 허용할지 정해줌. max_length= 범위 보다 큰(?) 예제는 배제됨.    
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples= 대신 example_selector= 지정
    example_selector=example_selector,

    suffix = "Human: What do you know about {country}?",
    input_variables=['country']
)


In [35]:
prompt.format(country='Brazil')   # 확인동작

'Human: What do you know about Brazil?'

In [ ]:
# ↑ example 들이 하나도 포맷팅이 안되어 있다??
# max_length 값이 너무 작아서 exmaple 하나도 포맷팅 안됨.

In [39]:
example_selector = LengthBasedExampleSelector(
    examples=examples, 
    example_prompt=example_prompt,
    max_length=80,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples= 대신 example_selector= 지정
    example_selector=example_selector,

    suffix = "Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country='Brazil'))

Human: France
AI:
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: What do you know about Brazil?


In [40]:
example_selector = LengthBasedExampleSelector(
    examples=examples, 
    example_prompt=example_prompt,
    max_length=180,  # max_length 값을 더 늘려보자
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    # examples= 대신 example_selector= 지정
    example_selector=example_selector,

    suffix = "Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country='Brazil'))

Human: France
AI:
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        

Human: Italy
AI:
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


## Custom ExampleSelector (BaseExampleSelector)

In [ ]:
# 내가 원하는 대로의 example 들을 허용할수 있도록 ExampleSelector 를 제공해줄수 있다

In [41]:
# BaseExampleSelector 를 상속 받아서 만든다.
from langchain_core.example_selectors.base import BaseExampleSelector

In [44]:
# BaseExampleSelector 의 구현객체를 만드려면
#  상속 받은뒤 select_examples() 과 add_example() 을 반드시 오버라이딩 해주어야 한다.

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    # select_examples()
    # 입력에 따라 어떠한 샘플을 사용할지 select 함.
    
    # 이번 예제에서는 examples 리스트 에서 random 으로 선택하게 하려 함.
    # ※ 이는 얼마든지 복잡하게 만들어 볼수도 있다.
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]  # examples 에서 무작위로 1개 선택한뒤 list 로 리턴

    # add_example()
    # Add new example to store.  이미 존재하는 example 에 example 을 추가하는 method
    def add_example(self, example):
        self.examples.append(example)  # 기존의 list 에 새로운 example 을 append
    

In [94]:
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,

    suffix = "Human: What do you know about {country}?",
    input_variables=['country']
)

print(prompt.format(country='Brazil'))

Human: Italy
AI:
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        

Human: What do you know about Brazil?


# 4.PromptTemplate 저장/읽기 load_prompt()

In [ ]:
# prompt 를 파일로 다룰 경우
#   'JSON' 파일  혹은 'YAML' 파일 로 만들수 있다


## json 파일

In [95]:
with open('prompt.json', 'w') as f:
    f.write("""
  {
    "_type":"prompt",
    "template":"What is the capital of {country}",
    "input_variables":["country"]
  }    
    """)

In [96]:
!type prompt.json


  {
    "_type":"prompt",
    "template":"What is the capital of {country}",
    "input_variables":["country"]
  }    
    


In [97]:
from langchain_core.prompts.loading import load_prompt

In [98]:
prompt = load_prompt('./prompt.json')

prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}')

In [99]:
prompt.format(country='Canada')

'What is the capital of Canada'

## yaml 파일

In [ ]:
# name: value
#  ★ name 은 쌍따옴표 없다.  : 다음에 한칸 꼭 띄우기!   뒤에 콤마 없다

In [100]:
with open("prompt.yaml", "w") as f:
    f.write("""
_type: "prompt"
template: "What is the capital of {country}"
input_variables: ["coutry"]
    """)

In [101]:
!type prompt.yaml


_type: "prompt"
template: "What is the capital of {country}"
input_variables: ["coutry"]
    


In [102]:
prompt = load_prompt("./prompt.yaml")
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='What is the capital of {country}')

In [103]:
prompt.format(country="Japan")

'What is the capital of Japan'

# 5.Caching

In [ ]:
# Caching 을 사용하면 모델의 응답을 저장할수 있다.
# 예를들어
# 똑같은 질문을 받는 상황이라면 그 때마다 답변생성할 필요 없이
# 이미 캐싱된 답변을 재사용 할수 있는 것이다 -->  비용절감! 

## set_llm_cache(), InMemoryCache

In [104]:
from langchain_core.globals import set_llm_cache

In [105]:
from langchain_core.caches import InMemoryCache

In [106]:
# ↓ 이렇게 캐시 세팅하면 LLM 의 모든 응답이 '메모리'에 저장된다.
set_llm_cache(InMemoryCache())

In [107]:
chat = ChatOpenAI(
    temperature=0.1
)

In [108]:
# 동일한 질문을 두번 해볼거다.  시간도 측정해보자.
import time
from datetime import timedelta

def check_laptime(message):
    start_time = time.time()
    response = chat.invoke(message)
    end_time = time.time()
    elapsed_time = end_time - start_time # 경과시간
    print('▶ 경과시간 %s' % (str(timedelta(seconds = elapsed_time))))
    print(f'{len(response.content)} 글자: {response.content}\n')



In [110]:
check_laptime("How do you make Italian pasta")

▶ 경과시간 0:00:03.478761
1009 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt
- Water (if needed)

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and make a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.
7. Cut the dough into your desired shape, such as fettuccine or spaghetti.
8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.
9. Drain the pasta and toss it with your favorite sauce or toppi

In [111]:
check_laptime("How do you make Italian pasta")

▶ 경과시간 0:00:00
1009 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- Pinch of salt
- Water (if needed)

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and make a well in the center.
2. Crack the eggs into the well and add a pinch of salt.
3. Using a fork, gradually mix the eggs into the flour until a dough forms.
4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.
5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.
6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.
7. Cut the dough into your desired shape, such as fettuccine or spaghetti.
8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.
9. Drain the pasta and toss it with your favorite sauce or toppings.

E

## set_debug()

In [112]:
from langchain_core.globals import set_debug

In [113]:
set_debug(True)

In [114]:
chat.invoke("How do you make Italian pasta")

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make Italian pasta"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the d

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of salted boiling water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or 

In [115]:
set_debug(False)

## SQLiteCache

In [116]:
from langchain_community.cache import SQLiteCache

In [117]:
set_llm_cache(SQLiteCache("cache.db"))

In [118]:
!dir

 D 드라이브의 볼륨: 새 볼륨
 볼륨 일련 번호: 02B1-19A2

 D:\LANG2509\LangWork\04_Model_IO 디렉터리

2025-11-05  오후 08:09    <DIR>          .
2025-11-05  오후 08:09    <DIR>          ..
2025-11-03  오후 08:46    <DIR>          .ipynb_checkpoints
2025-11-05  오후 08:08            60,450 04 Model IO.ipynb
2025-11-05  오후 08:09            32,768 cache.db
2025-11-05  오후 07:34               130 prompt.json
2025-11-05  오후 07:40                98 prompt.yaml
               4개 파일              93,446 바이트
               3개 디렉터리  455,918,051,328 바이트 남음


In [119]:
check_laptime("How do you make Italian pasta")  # 첫 실행

▶ 경과시간 0:00:03.619709
1124 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- 1/2 teaspoon of salt
- Water (if needed)

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and make a well in the center.
2. Crack the eggs into the well and add the salt.
3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.
4. Use your hands to knead the dough until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.
5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.
6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a rolling pin or pasta machine.
7. Cut the rolled-out dough into your desired pasta shape, such as fettuccine, spaghetti, or ravioli.
8. Cook the pasta in a large pot of boiling salted wate

In [120]:
# cache 된 답변을 사용
check_laptime("How do you make Italian pasta")

▶ 경과시간 0:00:00.001996
1124 글자: To make Italian pasta, you will need the following ingredients:

- 2 cups of all-purpose flour
- 2 large eggs
- 1/2 teaspoon of salt
- Water (if needed)

Here is a step-by-step guide to making Italian pasta:

1. On a clean work surface, pour the flour and make a well in the center.
2. Crack the eggs into the well and add the salt.
3. Using a fork, gradually mix the eggs into the flour until a dough starts to form.
4. Use your hands to knead the dough until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.
5. Once the dough is ready, cover it with a damp cloth and let it rest for about 30 minutes.
6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a rolling pin or pasta machine.
7. Cut the rolled-out dough into your desired pasta shape, such as fettuccine, spaghetti, or ravioli.
8. Cook the pasta in a large pot of boiling salted wate

https://reference.langchain.com/python/langchain_core/caches/

## debug, cache 사용을 끄려면

In [121]:
set_debug(False)
set_llm_cache(None)

# 6.Model config 저장,불러오기

## OpanAI 모델 호출 비용 확인하기

In [122]:
from langchain_community.callbacks.manager import get_openai_callback

In [123]:
with get_openai_callback() as usage:
    chat.invoke("What is the recipe for soju")
    print('🟦', usage)

🟦 Tokens Used: 222
	Prompt Tokens: 14
		Prompt Tokens Cached: 0
	Completion Tokens: 208
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.000319


In [124]:
with get_openai_callback() as usage:
    a = chat.invoke("What is the recipe for soju")
    b = chat.invoke("What is the recipe for bread")
    print('\n🐹', a.content)
    print('\n🐹', b.content)
    print('\n🟦', usage)


🐹 Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of yeast

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and steam it until fully cooked.
3. Let the rice cool down to room temperature.
4. In a large bowl, mix the nuruk and yeast with water until dissolved.
5. Add the cooked rice to the bowl and mix well.
6. Cover the bowl with a clean cloth and let it ferment in a warm place for 3-4 days.
7. After fermentation, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean container and let it sit for another 1-2 days to allow the flavors to develop.
9. Serve the homemade soju chilled and enjoy responsibly.

🐹 Ingredients:
- 4 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 2 teaspoons salt
- 2 tablespoons olive oil

Instructions:
1. In a large mixing bowl, combine the warm water, suga

In [125]:
print(usage.total_cost)
print(usage.total_tokens)

0.0007200000000000002
498


## Model config

In [126]:
from langchain_openai.llms.base import OpenAI

In [127]:
llm = OpenAI(
    temperature=0.1,
    max_tokens=450,  # 모델 I/O 에 소요되는 token 수 제한.
    model='gpt-4-turbo',  # 사용 모델 설정
)

In [128]:
# 모델 config 저장
llm.save('model.json')

In [129]:
!type model.json

{
    "model_name": "gpt-4-turbo",
    "temperature": 0.1,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "n": 1,
    "seed": null,
    "logprobs": null,
    "max_tokens": 450,
    "_type": "openai"
}


In [ ]:
# 저장된 모델 config 를 다시 읽어올수 있다

In [130]:
from langchain_community.llms.loading import load_llm

In [131]:
chat = load_llm('model.json')

D:\LANG2509\LangWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
D:\LANG2509\LangWork\.venv\Lib\site-packages\langchain_community\llms\openai.py:1089: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [134]:
import json
with open('model.json', 'r', encoding='utf-8') as f:
    config = json.load(f)

chat = OpenAI(**config)

D:\LANG2509\LangWork\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! _type is not default parameter.
                _type was transferred to model_kwargs.
                Please confirm that _type is what you intended.
  if await self.run_code(code, result, async_=asy):


In [135]:
chat

OpenAI(client=<openai.resources.completions.Completions object at 0x0000015514699460>, async_client=<openai.resources.completions.AsyncCompletions object at 0x0000015514765640>, model_name='gpt-4-turbo', temperature=0.1, max_tokens=450, model_kwargs={'_type': 'openai'}, openai_api_key=SecretStr('**********'))